In [1]:
def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0) + vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition) - 1):
        if partition[i] < partition[i + 1]:
            return {partition[0: i] + (partition[i] + partition[i + 1],)
                    + partition[i + 2: len(partition)]: partition[i + 1] - partition[i],
                    partition[0: i] + (partition[i + 1],) + (partition[i],)
                    + partition[i + 2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2: vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition + partition2: vector[partition2] for partition2 in set(vector)})

def to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return {tuple(list_of_partition[i]): row[i] for i in range(len(row)) if row[i] != 0}

def my_order(a):
    polynomial = to_polynomial(a)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(a), len(a) - number_of_ones(a), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def matrix_UVir_m2_m4(n):
    m2 = {(1, 1): -4, (2,): 3}
    m4 = {(4,): -36, (3, 1): 264, (2, 2): 49, (2, 1, 1): -600, (1, 1, 1, 1): 144}
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 4)
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition2[i]), m2)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m4 = [L_partition_vector(tuple(list_of_partition3[i]), m4)
                         for i in range(len(list_of_partition3))]
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m4 = matrix({(i, j): list_of_vector_m4[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m4)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m4(n)
    pivots = A_n.pivots()
    try:
        return to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

In [2]:
list_of_partition = [[2], [1, 1, 1, 1], [3, 1, 1, 1], [3, 3, 1], [4, 3, 1], [4, 4, 1, 1],
                     [5, 4, 1, 1, 1], [5, 5, 1, 1, 1], [6, 5, 3, 1, 1], [6, 6, 3, 1, 1],
                     [7, 6, 4, 1, 1, 1], [8, 7, 5, 3, 1, 1]]
%time mW_list_of_partition(list_of_partition)

CPU times: user 2min 40s, sys: 1.32 s, total: 2min 41s
Wall time: 2min 41s


[{(2,): 1, (1, 1): -4/3},
 {(1, 1, 1, 1): 1, (3, 1): -75/32, (4,): -465/256},
 {(3, 1, 1, 1): 1,
  (3, 3): -75/128,
  (4, 1, 1): -1/8,
  (5, 1): -111/64,
  (6,): -183/512},
 {(3, 3, 1): 1,
  (4, 1, 1, 1): 16/15,
  (4, 3): -3/10,
  (5, 1, 1): -6/5,
  (6, 1): -83/20,
  (7,): -13/20},
 {(4, 3, 1): 1,
  (4, 4): 1/8,
  (5, 3): 3/16,
  (6, 1, 1): -1,
  (7, 1): -37/16,
  (8,): -51/128},
 {(4, 4, 1, 1): 1,
  (5, 3, 1, 1): 4/3,
  (5, 4, 1): 6,
  (6, 3, 1): 21/8,
  (7, 1, 1, 1): -4,
  (5, 5): 193/256,
  (6, 4): 75/64,
  (7, 3): 285/128,
  (8, 1, 1): -41/3,
  (9, 1): -2895/128,
  (10,): -4203/1024},
 {(5, 4, 1, 1, 1): 1,
  (6, 3, 3): 81/256,
  (5, 5, 1, 1): 779/192,
  (6, 4, 1, 1): 13/2,
  (7, 3, 1, 1): 67/32,
  (6, 5, 1): 2295/128,
  (7, 4, 1): 351/32,
  (8, 3, 1): -27/8,
  (9, 1, 1, 1): -767/32,
  (6, 6): 1215/512,
  (7, 5): 7437/2048,
  (8, 4): -567/1024,
  (9, 3): 17253/2048,
  (10, 1, 1): -16189/256,
  (11, 1): -189243/2048,
  (12,): -291801/16384},
 {(5, 5, 1, 1, 1): 1,
  (6, 4, 1, 1, 1): 3